In [ ]:
#@title V1.0 Download file/Folder từ Google Drive URL by Andy N Le
#@markdown [![Badge with Link](https://img.shields.io/badge/DOWNLOAD-GOOGLE_DRIVE_URL-%2300A8E1?style=flat&logo=GITLAB&logoColor=84a805)](https://stablediffusion.sbs/)

# @markdown Chức năng download file/folder từ Link Goolgle (public link nha) về Google drive của mình.
# @markdown Lưu ý: nhớ donate cho tác giả nha. Momo/zalo/vnpay: 0908231181

%cd /content
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Bước 2: Thiết lập các tham số
# URL thư mục Google Drive để sao chép
gDrive_url = 'https://drive.google.com/drive/folders/1MZGN7byAmraGQ3Hl_I8gRiLl-VeOQzT8'  # @param {type: 'string'}

# Thư mục đích (đường dẫn đích)
Target_Folder = 'https://drive.google.com/drive/folders/16_dNTTZqkdULJssBW-XVsMM2EtJE4ve4?usp=drive_link'  # @param {type: 'string'}

# Phạm vi trang
from_page = 0  # Trang bắt đầu (số nguyên) @param {type: "integer"}
to_page = 0    # Trang kết thúc (số nguyên) @param {type: "integer"}

#@markdown Kích thước tải xuống tối đa (GB) :
max_download_size_input = '1000'  # @param {type: "string"}
try:
    max_download_size = float(max_download_size_input)
except ValueError:
    print("Lỗi: max_download_size phải là một giá trị số.")
    raise ValueError("Nhập max_download_size không hợp lệ. Vui lòng nhập một số.")

# Chuỗi loại trừ (lọc tệp/thư mục theo tên)
exclude_str = ""  # Loại bỏ tệp hoặc thư mục có chứa các chuỗi này, phân tách bằng dấu phẩy @param {type: "string"}

# Bước 3: Import thư viện và định nghĩa lớp trợ giúp
import os
import time
import re
import sys
from googleapiclient.discovery import build
from google.colab import auth
from googleapiclient.errors import HttpError

class DownloadFromDrive:
    def __init__(self):
        self._total_size = 0
        self._limit_size = max_download_size
        self.excluded_strings = [ext.strip() for ext in exclude_str.split(",") if ext.strip()]

    def get_user_credential(self):
        auth.authenticate_user()
        drive_service = build('drive', 'v3')
        return drive_service

    def get_childs_from_folder(self, drive_service, folder_id, from_page, to_page):
        files = []
        page_token = None
        query = f"'{folder_id}' in parents and trashed = false"
        if self.excluded_strings:
            query += " and " + " and ".join([f"not name contains '{ext}'" for ext in self.excluded_strings])

        pages = 0
        while True:
            pages += 1
            response = drive_service.files().list(q=query, orderBy='name, createdTime',
                                                  fields='files(id, name, mimeType, size), nextPageToken',
                                                  pageToken=page_token, supportsAllDrives=True,
                                                  includeItemsFromAllDrives=True).execute()
            if (from_page < pages <= to_page) or to_page == 0:
                files.extend(response.get('files', []))
            page_token = response.get('nextPageToken', None)
            if page_token is None or (pages >= to_page > 0):
                break

        print(f"Tổng số tệp: {len(files)}")
        return files

    def copy_file(self, drive_service, dest_folder_id, source_file):
        if source_file['mimeType'] != 'application/vnd.google-apps.folder':
            body_file_inf = {'parents': [dest_folder_id]}
            if not self.check_if_exists(drive_service, dest_folder_id, source_file['name']):
                try:
                    start_time = time.time()
                    drive_service.files().copy(body=body_file_inf, fileId=source_file['id'], supportsAllDrives=True).execute()
                    end_time = time.time()
                    fileSize = int(source_file.get('size', 0))
                    size_mb = fileSize / (1024 * 1024)
                    self._total_size += size_mb
                    speed_mb = size_mb / (end_time - start_time)
                    print(f"[{source_file['name']}] đã sao chép. Kích thước {size_mb:0.2f} MB. Tốc độ {speed_mb:0.2f} MB/s")

                    if self._total_size >= (self._limit_size * 1024):
                        self.on_total_size_exceeded(f"Tổng dung lượng vượt quá {self._limit_size} GB. Kết thúc chương trình.")
                except Exception as e:
                    print("Đã xảy ra lỗi: ", e)
            else:
                print(f"[{source_file['name']}] đã tồn tại.")
        else:
            source_files = self.get_childs_from_folder(drive_service, source_file['id'], 0, 0)
            if source_files:
                print(f"Sao chép trong thư mục {source_file['name']} bắt đầu")
                sub_folder_id = self.create_folder(drive_service, dest_folder_id, source_file['name'])
                self.copy_multiple_files(drive_service, sub_folder_id, source_files)
                print(f"Sao chép trong thư mục {source_file['name']} kết thúc")

    def create_folder(self, drive_service, dest_folder_id, sub_folder_name):
        sub_folder_inf = {'name': sub_folder_name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [dest_folder_id]}
        exist_folder_id = self.check_if_exists(drive_service, dest_folder_id, sub_folder_name)
        if not exist_folder_id:
            try:
                folder = drive_service.files().create(body=sub_folder_inf, fields='id').execute()
                return folder['id']
            except Exception as e:
                print("Đã xảy ra lỗi: ", e)
        return exist_folder_id

    def check_if_exists(self, drive_service, dest_folder_id, name):
        try:
            processed_name = name.replace("'", "\\'")
            results = drive_service.files().list(q=f"'{dest_folder_id}' in parents and name contains '{processed_name}' and trashed=false",
                                                fields='files(id)').execute()
            if 'files' in results and results['files']:
                return results['files'][0]['id']
        except Exception as e:
            print("Đã xảy ra lỗi: ", e)
        return ""

    def copy_multiple_files(self, drive_service, dest_folder_id, source_files):
        for source_file in source_files:
            self.copy_file(drive_service, dest_folder_id, source_file)

    def extract_folder_id_from_url(self, url):
        pattern = r'[-\w]{25,}'
        match = re.search(pattern, url)
        if match:
            folder_id = match.group(0)
            try:
                drive_service = self.get_user_credential()  # Đảm bảo API Drive đã xác thực
                drive_service.files().get(fileId=folder_id, supportsAllDrives=True).execute()
                return folder_id
            except HttpError:
                print(f"Lỗi: Không thể truy cập thư mục với ID {folder_id}. Kiểm tra quyền và URL.")
                return None
        else:
            print("Lỗi: Định dạng URL không hợp lệ.")
            return None

    def on_total_size_exceeded(self, message):
        print(message)
        sys.exit()

    def copy_drive_to_drive(self, destDriveLink, sourceDriveLink, from_page, to_page):
        service = self.get_user_credential()
        start_time = time.time()
        dest_folder_id = self.extract_folder_id_from_url(destDriveLink)
        source_folder_id = self.extract_folder_id_from_url(sourceDriveLink)
        if not dest_folder_id or not source_folder_id:
            print("Không thể tiếp tục do lỗi truy cập thư mục.")
            return
        source_folder = service.files().get(fileId=source_folder_id, supportsAllDrives=True).execute()
        new_dest_folder_id = self.create_folder(service, dest_folder_id, source_folder['name'])
        source_files = self.get_childs_from_folder(service, source_folder_id, from_page, to_page)
        self.copy_multiple_files(service, new_dest_folder_id, source_files)
        end_time = time.time()
        size_gb = self._total_size / 1024
        speed_mb = self._total_size / (end_time - start_time)
        print(f"Xong. Tổng kích thước {size_gb:0.2f} GB. Thời gian {int(end_time - start_time)} giây. Tốc độ {speed_mb:0.2f} MB/s")

        # In đường link đến thư mục đích đã sao chép
        destination_link = f"https://drive.google.com/drive/folders/{new_dest_folder_id}"
        print(f"Thư mục đã sao chép vào: {destination_link}")

# Chạy chính
downloader = DownloadFromDrive()
downloader.copy_drive_to_drive(Target_Folder, gDrive_url, from_page, to_page)


/content
Mounted at /content/gdrive
Tổng số tệp: 276
[_BMT7810.ARW] đã sao chép. Kích thước 23.79 MB. Tốc độ 7.85 MB/s
[_BMT7810.JPG] đã sao chép. Kích thước 11.50 MB. Tốc độ 6.80 MB/s
[_BMT7811.ARW] đã sao chép. Kích thước 23.80 MB. Tốc độ 8.10 MB/s
[_BMT7811.JPG] đã sao chép. Kích thước 11.75 MB. Tốc độ 6.58 MB/s
[_BMT7812.ARW] đã sao chép. Kích thước 23.80 MB. Tốc độ 9.01 MB/s
[_BMT7812.JPG] đã sao chép. Kích thước 11.75 MB. Tốc độ 6.29 MB/s
[_BMT7813.ARW] đã sao chép. Kích thước 23.78 MB. Tốc độ 9.89 MB/s
[_BMT7813.JPG] đã sao chép. Kích thước 11.31 MB. Tốc độ 5.31 MB/s
[_BMT7814.ARW] đã sao chép. Kích thước 23.78 MB. Tốc độ 8.57 MB/s
[_BMT7814.JPG] đã sao chép. Kích thước 11.31 MB. Tốc độ 5.92 MB/s
[_BMT7815.ARW] đã sao chép. Kích thước 23.78 MB. Tốc độ 9.27 MB/s
[_BMT7815.JPG] đã sao chép. Kích thước 11.38 MB. Tốc độ 5.84 MB/s
[_BMT7816.ARW] đã sao chép. Kích thước 23.78 MB. Tốc độ 9.72 MB/s
[_BMT7816.JPG] đã sao chép. Kích thước 11.41 MB. Tốc độ 5.56 MB/s
[_BMT7817.ARW] đã sao c

**HƯỚNG DẪN: **
- Nhập link google Drive của người khác mà bạn muốn tải vào ô: **gDrive_url**
- Nhập link thư mục google Drive mà muốn tải về vào ô: **Target_Folder**
- Nhập dung lượng file/thư mục tối đa cho 1 lần tải về vào ô: **max_download_size_input**
- Nhấn **icon** (▶) bên trái để chạy chương trình
- Nhớ donate cho tác giả: **Momo/zalo/VNPay: 0908231181**